In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import accuracy_score, precision_score, f1_score
from scipy.signal import resample

# 数据增强 - 添加噪声
def add_noise(data, noise_factor=0.01):
    noise = noise_factor * np.random.normal(loc=0.0, scale=1.0, size=data.shape)
    return data + noise

# 数据增强 - 时间平移
def time_shift(data, shift_max=5):
    shift = np.random.randint(-shift_max, shift_max)
    return np.roll(data, shift, axis=0)

# 数据增强 - 时间缩放
def time_stretch(data, stretch_factor=0.8):
    length = int(data.shape[0] * stretch_factor)
    data = resample(data, length)
    if length != 50:  # 强制重采样为50个点
        data = resample(data, 50)
    return data

# 综合数据增强函数
def augment_data(data):
    # 确保数据是二维的，每行对应一个时间步，列对应不同的通道
    assert data.shape == (50, 3), f"数据形状不正确，期望 (50, 3)，但得到了 {data.shape}"
    
    augmented_data = np.empty_like(data)  # 初始化增强后的数据
    for i in range(3):  # 遍历每个通道 (x, y, z)
        channel_data = data[:, i]  # 提取单个通道的数据
        channel_data = add_noise(channel_data)
        channel_data = time_shift(channel_data)
        channel_data = time_stretch(channel_data, stretch_factor=0.8)
        augmented_data[:, i] = channel_data  # 将增强后的通道数据放回去
    return augmented_data

# 自定义数据集，包含数据增强
class AugmentedDataset(Dataset):
    def __init__(self, data, labels_defect, labels_severity, augment=True):
        self.data = data
        self.labels_defect = labels_defect
        self.labels_severity = labels_severity
        self.augment = augment

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx].reshape(50, 3)  # 将每个样本 reshape 成 (50, 3)
        label_defect = self.labels_defect[idx]
        label_severity = self.labels_severity[idx]
    
         # 如果 augment=True，进行数据增强
        if self.augment:
          sample = augment_data(sample)

        return torch.tensor(sample, dtype=torch.float32), torch.tensor(label_defect, dtype=torch.long), torch.tensor(label_severity, dtype=torch.long)

# 1. 加载数据
dataX = np.loadtxt('dataset1/dataX.txt')  # 每组数据50个点
dataY = np.loadtxt('dataset1/dataY.txt')
dataZ = np.loadtxt('dataset1/dataZ.txt')
dataLabels = np.loadtxt('dataset1/dataLabel.txt')  # 假设每一行有两列

# 将标签分为两个部分：defect 标签和 severity 标签
labels_defect = dataLabels[:, 0].astype(int)  # 道路缺陷标签
labels_severity = dataLabels[:, 1].astype(int)  # 严重程度标签

# 2. 数据预处理：合并 x, y, z 方向的数据
data = np.column_stack((dataX, dataY, dataZ))  # [样本数, 150]

# 3. 划分数据集
X_train, X_test, y_train_defect, y_test_defect, y_train_severity, y_test_severity = train_test_split(
    data, labels_defect, labels_severity, test_size=0.2, stratify=labels_defect, random_state=42
)

# 创建数据加载器时应用增强数据集
train_dataset = AugmentedDataset(X_train, y_train_defect, y_train_severity, augment=True)
test_dataset = AugmentedDataset(X_test, y_test_defect, y_test_severity, augment=False)

train_loader = DataLoader(train_dataset, batch_size=5120, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=5120, shuffle=False)

class Simplified_CNN_Model(nn.Module):
    def __init__(self):
        super(Simplified_CNN_Model, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=3, out_channels=16, kernel_size=5, stride=1, padding=2)
        self.bn1 = nn.BatchNorm1d(16)
        self.pool1 = nn.MaxPool1d(kernel_size=2, stride=2)

        self.conv2 = nn.Conv1d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm1d(32)
        self.pool2 = nn.MaxPool1d(kernel_size=2, stride=2)

        self.fc1 = nn.Linear(32 * 12, 64)  # Adjusted input size after pooling
        self.dropout = nn.Dropout(0.3)  # Reduced dropout to allow more learning
        self.fc2 = nn.Linear(64, 32)

        self.fc_defect = nn.Linear(32, 3)  # Defect classification (3 classes)
        self.fc_severity = nn.Linear(32, 4)  # Severity classification (4 classes)

    def forward(self, x):
        x = x.view(-1, 50, 3).permute(0, 2, 1)

        x = self.pool1(torch.relu(self.bn1(self.conv1(x))))
        x = self.pool2(torch.relu(self.bn2(self.conv2(x))))

        x = x.view(-1, 32 * 12)  # Adjust size based on the layer output
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = torch.relu(self.fc2(x))

        defect_output = self.fc_defect(x)
        severity_output = self.fc_severity(x)

        return defect_output, severity_output


# 创建模型实例
model = Simplified_CNN_Model()

# 损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 训练模型
def train_model(model, train_loader, criterion, optimizer, num_epochs=20):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, defect_labels, severity_labels in train_loader:
            optimizer.zero_grad()
            defect_preds, severity_preds = model(inputs)
            loss_defect = criterion(defect_preds, defect_labels)
            loss_severity = criterion(severity_preds, severity_labels)
            loss = loss_defect + loss_severity
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss / len(train_loader)}')

# 测试模型
def test_model(model, test_loader):
    model.eval()
    all_defect_labels = []
    all_defect_preds = []
    all_severity_labels = []
    all_severity_preds = []
    
    with torch.no_grad():
        for inputs, defect_labels, severity_labels in test_loader:
            defect_preds, severity_preds = model(inputs)
            _, predicted_defects = torch.max(defect_preds, 1)
            _, predicted_severities = torch.max(severity_preds, 1)

            all_defect_labels.extend(defect_labels.cpu().numpy())
            all_defect_preds.extend(predicted_defects.cpu().numpy())
            all_severity_labels.extend(severity_labels.cpu().numpy())
            all_severity_preds.extend(predicted_severities.cpu().numpy())

    accuracy_defect = accuracy_score(all_defect_labels, all_defect_preds)
    precision_defect = precision_score(all_defect_labels, all_defect_preds, average='weighted', zero_division=0)
    f1_defect = f1_score(all_defect_labels, all_defect_preds, average='weighted', zero_division=0)

    accuracy_severity = accuracy_score(all_severity_labels, all_severity_preds)
    precision_severity = precision_score(all_severity_labels, all_severity_preds, average='weighted', zero_division=0)
    f1_severity = f1_score(all_severity_labels, all_severity_preds, average='weighted', zero_division=0)

    print(f'道路缺陷分类准确率: {accuracy_defect * 100:.2f}%')
    print(f'严重程度分类准确率: {accuracy_severity * 100:.2f}%')
    print(f'道路缺陷分类精确度: {precision_defect:.2f}')
    print(f'严重程度分类精确度: {precision_severity:.2f}')
    print(f'道路缺陷分类F1值: {f1_defect:.2f}')
    print(f'严重程度分类F1值: {f1_severity:.2f}')

# 开始训练和测试
train_model(model, train_loader, criterion, optimizer, num_epochs=200)
test_model(model, test_loader)


In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import accuracy_score, precision_score, f1_score
from scipy.signal import resample
import optuna  # 确保已安装 optuna 库

# 数据增强函数
def add_noise(data, noise_factor=0.01):
    noise = noise_factor * np.random.normal(loc=0.0, scale=1.0, size=data.shape)
    return data + noise

def time_shift(data, shift_max=5):
    shift = np.random.randint(-shift_max, shift_max)
    return np.roll(data, shift, axis=0)

def time_stretch(data, stretch_factor=0.8):
    length = int(data.shape[0] * stretch_factor)
    data = resample(data, length)
    if length != 50:  # 强制重采样为50个点
        data = resample(data, 50)
    return data

def augment_data(data):
    # 确保输入数据形状为 (50, 3)
    data = data.reshape(50, 3)
    augmented_data = np.empty_like(data)
    for i in range(3):  # 对每个通道进行增强
        channel_data = data[:, i]
        channel_data = add_noise(channel_data)
        channel_data = time_shift(channel_data)
        channel_data = time_stretch(channel_data, stretch_factor=0.8)
        augmented_data[:, i] = channel_data
    return augmented_data

# 自定义数据集，包含数据增强
class AugmentedDataset(Dataset):
    def __init__(self, data, labels_defect, labels_severity, augment=True):
        self.data = data
        self.labels_defect = labels_defect
        self.labels_severity = labels_severity
        self.augment = augment

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx]
        label_defect = self.labels_defect[idx]
        label_severity = self.labels_severity[idx]

        # 如果 augment=True，进行数据增强
        if self.augment:
            sample = augment_data(sample)
        else:
            sample = sample.reshape(50, 3)  # 保证形状一致

        return torch.tensor(sample, dtype=torch.float32), torch.tensor(label_defect, dtype=torch.long), torch.tensor(label_severity, dtype=torch.long)

# 加载数据
dataX = np.loadtxt('dataset1/dataX.txt')  # 每组数据50个点
dataY = np.loadtxt('dataset1/dataY.txt')
dataZ = np.loadtxt('dataset1/dataZ.txt')
dataLabels = np.loadtxt('dataset1/dataLabel.txt')  # 假设每一行有两列

# 将标签分为两个部分：defect 标签和 severity 标签
labels_defect = dataLabels[:, 0].astype(int)  # 道路缺陷标签
labels_severity = dataLabels[:, 1].astype(int)  # 严重程度标签

# 数据预处理：合并 x, y, z 方向的数据
data = np.hstack((dataX, dataY, dataZ))  # [样本数, 150]

# 划分数据集
X_train_val, X_test, y_train_val_defect, y_test_defect, y_train_val_severity, y_test_severity = train_test_split(
    data, labels_defect, labels_severity, test_size=0.2, stratify=labels_defect, random_state=42
)

def objective(trial):
    # 定义需要优化的超参数
    learning_rate = trial.suggest_loguniform('lr', 1e-4, 1e-2)
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128])
    dropout_rate = trial.suggest_uniform('dropout_rate', 0.2, 0.5)
    num_filters1 = trial.suggest_categorical('num_filters1', [16, 32, 64])
    num_filters2 = trial.suggest_categorical('num_filters2', [32, 64, 128])

    # 划分训练集和验证集
    X_train, X_val, y_train_defect, y_val_defect, y_train_severity, y_val_severity = train_test_split(
        X_train_val, y_train_val_defect, y_train_val_severity, test_size=0.2, stratify=y_train_val_defect, random_state=42
    )

    # 创建数据加载器
    train_dataset = AugmentedDataset(X_train, y_train_defect, y_train_severity, augment=True)
    val_dataset = AugmentedDataset(X_val, y_val_defect, y_val_severity, augment=False)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    # 定义模型
    class CNN_Model(nn.Module):
        def __init__(self):
            super(CNN_Model, self).__init__()
            self.conv1 = nn.Conv1d(in_channels=3, out_channels=num_filters1, kernel_size=5, stride=1, padding=2)
            self.bn1 = nn.BatchNorm1d(num_filters1)
            self.pool1 = nn.MaxPool1d(kernel_size=2, stride=2)

            self.conv2 = nn.Conv1d(in_channels=num_filters1, out_channels=num_filters2, kernel_size=3, stride=1, padding=1)
            self.bn2 = nn.BatchNorm1d(num_filters2)
            self.pool2 = nn.MaxPool1d(kernel_size=2, stride=2)

            self.fc1 = nn.Linear(num_filters2 * 12, 64)
            self.dropout = nn.Dropout(dropout_rate)
            self.fc2 = nn.Linear(64, 32)

            self.fc_defect = nn.Linear(32, 3)  # 道路缺陷分类 (3 类)
            self.fc_severity = nn.Linear(32, 4)  # 严重程度分类 (4 类)

        def forward(self, x):
            x = x.permute(0, 2, 1)  # [batch_size, 3, 50]

            x = self.pool1(torch.relu(self.bn1(self.conv1(x))))
            x = self.pool2(torch.relu(self.bn2(self.conv2(x))))

            x = x.view(x.size(0), -1)
            x = torch.relu(self.fc1(x))
            x = self.dropout(x)
            x = torch.relu(self.fc2(x))

            defect_output = self.fc_defect(x)
            severity_output = self.fc_severity(x)

            return defect_output, severity_output

    model = CNN_Model()

    # 定义损失函数和优化器
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # 训练模型
    num_epochs = 10  # 为了加快优化过程，设置较小的 epoch 数
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, defect_labels, severity_labels in train_loader:
            optimizer.zero_grad()
            defect_preds, severity_preds = model(inputs)
            loss_defect = criterion(defect_preds, defect_labels)
            loss_severity = criterion(severity_preds, severity_labels)
            loss = loss_defect + loss_severity
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

    # 在验证集上评估模型
    model.eval()
    all_defect_labels = []
    all_defect_preds = []
    all_severity_labels = []
    all_severity_preds = []

    with torch.no_grad():
        for inputs, defect_labels, severity_labels in val_loader:
            defect_preds, severity_preds = model(inputs)
            _, predicted_defects = torch.max(defect_preds, 1)
            _, predicted_severities = torch.max(severity_preds, 1)

            all_defect_labels.extend(defect_labels.cpu().numpy())
            all_defect_preds.extend(predicted_defects.cpu().numpy())
            all_severity_labels.extend(severity_labels.cpu().numpy())
            all_severity_preds.extend(predicted_severities.cpu().numpy())

    # 计算验证集上的准确率
    accuracy_defect = accuracy_score(all_defect_labels, all_defect_preds)
    accuracy_severity = accuracy_score(all_severity_labels, all_severity_preds)

    # 返回验证集上平均准确率的负值，Optuna 默认最小化目标
    avg_accuracy = (accuracy_defect + accuracy_severity) / 2
    return -avg_accuracy

# 创建 Optuna study 并开始优化
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=20)

# 输出最佳超参数
print('最佳超参数: ')
print(study.best_params)

# 使用最佳超参数重新训练模型并在测试集上评估
best_params = study.best_params
learning_rate = best_params['lr']
batch_size = best_params['batch_size']
dropout_rate = best_params['dropout_rate']
num_filters1 = best_params['num_filters1']
num_filters2 = best_params['num_filters2']

# 创建数据加载器
train_dataset = AugmentedDataset(X_train_val, y_train_val_defect, y_train_val_severity, augment=True)
test_dataset = AugmentedDataset(X_test, y_test_defect, y_test_severity, augment=False)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# 定义模型
class CNN_Model(nn.Module):
    def __init__(self):
        super(CNN_Model, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=3, out_channels=num_filters1, kernel_size=5, stride=1, padding=2)
        self.bn1 = nn.BatchNorm1d(num_filters1)
        self.pool1 = nn.MaxPool1d(kernel_size=2, stride=2)

        self.conv2 = nn.Conv1d(in_channels=num_filters1, out_channels=num_filters2, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm1d(num_filters2)
        self.pool2 = nn.MaxPool1d(kernel_size=2, stride=2)

        self.fc1 = nn.Linear(num_filters2 * 12, 64)
        self.dropout = nn.Dropout(dropout_rate)
        self.fc2 = nn.Linear(64, 32)

        self.fc_defect = nn.Linear(32, 3)  # 道路缺陷分类 (3 类)
        self.fc_severity = nn.Linear(32, 4)  # 严重程度分类 (4 类)

    def forward(self, x):
        x = x.permute(0, 2, 1)  # [batch_size, 3, 50]

        x = self.pool1(torch.relu(self.bn1(self.conv1(x))))
        x = self.pool2(torch.relu(self.bn2(self.conv2(x))))

        x = x.view(x.size(0), -1)
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = torch.relu(self.fc2(x))

        defect_output = self.fc_defect(x)
        severity_output = self.fc_severity(x)

        return defect_output, severity_output

model = CNN_Model()

# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# 训练模型
num_epochs = 20
model.train()
for epoch in range(num_epochs):
    running_loss = 0.0
    for inputs, defect_labels, severity_labels in train_loader:
        optimizer.zero_grad()
        defect_preds, severity_preds = model(inputs)
        loss_defect = criterion(defect_preds, defect_labels)
        loss_severity = criterion(severity_preds, severity_labels)
        loss = loss_defect + loss_severity
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss / len(train_loader)}')

# 测试模型
model.eval()
all_defect_labels = []
all_defect_preds = []
all_severity_labels = []
all_severity_preds = []

with torch.no_grad():
    for inputs, defect_labels, severity_labels in test_loader:
        defect_preds, severity_preds = model(inputs)
        _, predicted_defects = torch.max(defect_preds, 1)
        _, predicted_severities = torch.max(severity_preds, 1)

        all_defect_labels.extend(defect_labels.cpu().numpy())
        all_defect_preds.extend(predicted_defects.cpu().numpy())
        all_severity_labels.extend(severity_labels.cpu().numpy())
        all_severity_preds.extend(predicted_severities.cpu().numpy())

# 计算测试集上的指标
accuracy_defect = accuracy_score(all_defect_labels, all_defect_preds)
precision_defect = precision_score(all_defect_labels, all_defect_preds, average='weighted', zero_division=0)
f1_defect = f1_score(all_defect_labels, all_defect_preds, average='weighted', zero_division=0)

accuracy_severity = accuracy_score(all_severity_labels, all_severity_preds)
precision_severity = precision_score(all_severity_labels, all_severity_preds, average='weighted', zero_division=0)
f1_severity = f1_score(all_severity_labels, all_severity_preds, average='weighted', zero_division=0)

print(f'道路缺陷分类准确率: {accuracy_defect * 100:.2f}%')
print(f'严重程度分类准确率: {accuracy_severity * 100:.2f}%')
print(f'道路缺陷分类精确度: {precision_defect:.2f}')
print(f'严重程度分类精确度: {precision_severity:.2f}')
print(f'道路缺陷分类F1值: {f1_defect:.2f}')
print(f'严重程度分类F1值: {f1_severity:.2f}')


[I 2024-09-10 21:01:20,930] Finished trial#0 resulted in value: -0.605. Current best value is -0.605 with parameters: {'lr': 0.002298583419284908, 'batch_size': 32, 'dropout_rate': 0.3865475393880122, 'num_filters1': 32, 'num_filters2': 64}.
[I 2024-09-10 21:01:22,482] Finished trial#1 resulted in value: -0.63. Current best value is -0.63 with parameters: {'lr': 0.0009387583095713397, 'batch_size': 128, 'dropout_rate': 0.25203985525161676, 'num_filters1': 64, 'num_filters2': 32}.
[I 2024-09-10 21:01:24,937] Finished trial#2 resulted in value: -0.61. Current best value is -0.63 with parameters: {'lr': 0.0009387583095713397, 'batch_size': 128, 'dropout_rate': 0.25203985525161676, 'num_filters1': 64, 'num_filters2': 32}.
[I 2024-09-10 21:01:26,478] Finished trial#3 resulted in value: -0.61. Current best value is -0.63 with parameters: {'lr': 0.0009387583095713397, 'batch_size': 128, 'dropout_rate': 0.25203985525161676, 'num_filters1': 64, 'num_filters2': 32}.
[I 2024-09-10 21:01:28,904] F

最佳超参数: 
{'lr': 0.0005419731824757451, 'batch_size': 128, 'dropout_rate': 0.4572632981397161, 'num_filters1': 64, 'num_filters2': 128}
Epoch [1/20], Loss: 2.2496799677610397
Epoch [2/20], Loss: 1.788940668106079
Epoch [3/20], Loss: 1.6425750255584717
Epoch [4/20], Loss: 1.533383309841156
Epoch [5/20], Loss: 1.4740909039974213
Epoch [6/20], Loss: 1.4282962828874588
Epoch [7/20], Loss: 1.3928359746932983
Epoch [8/20], Loss: 1.3654481768608093
Epoch [9/20], Loss: 1.2847770005464554
Epoch [10/20], Loss: 1.303515538573265
Epoch [11/20], Loss: 1.301405742764473
Epoch [12/20], Loss: 1.2828572392463684
Epoch [13/20], Loss: 1.2489109337329865
Epoch [14/20], Loss: 1.2355439141392708
Epoch [15/20], Loss: 1.2427116930484772
Epoch [16/20], Loss: 1.2360970228910446
Epoch [17/20], Loss: 1.2608822733163834
Epoch [18/20], Loss: 1.2593141347169876
Epoch [19/20], Loss: 1.2058682143688202
Epoch [20/20], Loss: 1.1775417625904083
道路缺陷分类准确率: 57.20%
严重程度分类准确率: 48.40%
道路缺陷分类精确度: 0.75
严重程度分类精确度: 0.69
道路缺陷分类F1值: 

In [4]:
def objective(trial):
    # 优化超参数
    learning_rate = trial.suggest_loguniform('lr', 1e-5, 1e-2)
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64, 128])
    dropout_rate = trial.suggest_uniform('dropout_rate', 0.1, 0.5)
    num_filters1 = trial.suggest_categorical('num_filters1', [32, 64, 128])
    num_filters2 = trial.suggest_categorical('num_filters2', [64, 128, 256])
  
    # 划分训练集和验证集
    X_train, X_val, y_train_defect, y_val_defect, y_train_severity, y_val_severity = train_test_split(
        X_train_val, y_train_val_defect, y_train_val_severity, test_size=0.2, stratify=y_train_val_defect, random_state=42
    )

    # 创建数据加载器
    train_dataset = AugmentedDataset(X_train, y_train_defect, y_train_severity, augment=True)
    val_dataset = AugmentedDataset(X_val, y_val_defect, y_val_severity, augment=False)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    # 定义模型
    class CNN_Model(nn.Module):
        def __init__(self):
            super(CNN_Model, self).__init__()
            self.conv1 = nn.Conv1d(in_channels=3, out_channels=num_filters1, kernel_size=5, stride=1, padding=2)
            self.bn1 = nn.BatchNorm1d(num_filters1)
            self.pool1 = nn.MaxPool1d(kernel_size=2, stride=2)

            self.conv2 = nn.Conv1d(in_channels=num_filters1, out_channels=num_filters2, kernel_size=3, stride=1, padding=1)
            self.bn2 = nn.BatchNorm1d(num_filters2)
            self.pool2 = nn.MaxPool1d(kernel_size=2, stride=2)

            self.fc1 = nn.Linear(num_filters2 * 12, 128)  # 调整全连接层大小
            self.dropout = nn.Dropout(dropout_rate)
            self.fc2 = nn.Linear(128, 64)  # 增加复杂度

            self.fc_defect = nn.Linear(64, 3)  # 道路缺陷分类 (3 类)
            self.fc_severity = nn.Linear(64, 4)  # 严重程度分类 (4 类)

        def forward(self, x):
            x = x.permute(0, 2, 1)

            x = self.pool1(torch.relu(self.bn1(self.conv1(x))))
            x = self.pool2(torch.relu(self.bn2(self.conv2(x))))

            x = x.view(x.size(0), -1)
            x = torch.relu(self.fc1(x))
            x = self.dropout(x)
            x = torch.relu(self.fc2(x))

            defect_output = self.fc_defect(x)
            severity_output = self.fc_severity(x)

            return defect_output, severity_output

    model = CNN_Model()

    # 定义损失函数和优化器
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # 训练模型
    num_epochs = 20  # 增加训练 epoch
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, defect_labels, severity_labels in train_loader:
            optimizer.zero_grad()
            defect_preds, severity_preds = model(inputs)
            loss_defect = criterion(defect_preds, defect_labels)
            loss_severity = criterion(severity_preds, severity_labels)
            loss = loss_defect + loss_severity
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

    # 在验证集上评估模型
    model.eval()
    all_defect_labels = []
    all_defect_preds = []
    all_severity_labels = []
    all_severity_preds = []

    with torch.no_grad():
        for inputs, defect_labels, severity_labels in val_loader:
            defect_preds, severity_preds = model(inputs)
            _, predicted_defects = torch.max(defect_preds, 1)
            _, predicted_severities = torch.max(severity_preds, 1)

            all_defect_labels.extend(defect_labels.cpu().numpy())
            all_defect_preds.extend(predicted_defects.cpu().numpy())
            all_severity_labels.extend(severity_labels.cpu().numpy())
            all_severity_preds.extend(predicted_severities.cpu().numpy())

    # 计算验证集上的准确率
    accuracy_defect = accuracy_score(all_defect_labels, all_defect_preds)
    accuracy_severity = accuracy_score(all_severity_labels, all_severity_preds)

    avg_accuracy = (accuracy_defect + accuracy_severity) / 2
    return -avg_accuracy

# 调整搜索范围后，重新运行
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=30)  # 增加试验次数

# 最佳超参数结果
print('最佳超参数: ', study.best_params)


[I 2024-09-10 21:10:22,722] Finished trial#0 resulted in value: -0.65. Current best value is -0.65 with parameters: {'lr': 0.0001555203402272037, 'batch_size': 128, 'dropout_rate': 0.1933772703266984, 'num_filters1': 128, 'num_filters2': 128}.
[I 2024-09-10 21:10:28,948] Finished trial#1 resulted in value: -0.6950000000000001. Current best value is -0.6950000000000001 with parameters: {'lr': 1.2518519791871144e-05, 'batch_size': 32, 'dropout_rate': 0.28146522804254986, 'num_filters1': 128, 'num_filters2': 256}.
[I 2024-09-10 21:10:34,470] Finished trial#2 resulted in value: -0.7050000000000001. Current best value is -0.7050000000000001 with parameters: {'lr': 1.359618913218973e-05, 'batch_size': 32, 'dropout_rate': 0.2943027572468289, 'num_filters1': 128, 'num_filters2': 128}.
[I 2024-09-10 21:10:41,810] Finished trial#3 resulted in value: -0.7275. Current best value is -0.7275 with parameters: {'lr': 0.0036299288393169594, 'batch_size': 16, 'dropout_rate': 0.4673657097607763, 'num_fil

最佳超参数:  {'lr': 0.007916832081882586, 'batch_size': 16, 'dropout_rate': 0.4983031929840477, 'num_filters1': 32, 'num_filters2': 64}


In [9]:
def train_and_test_with_best_params(best_params):
    # 从最佳参数中提取超参数
    learning_rate = best_params['lr']
    batch_size = best_params['batch_size']
    dropout_rate = best_params['dropout_rate']
    num_filters1 = best_params['num_filters1']
    num_filters2 = best_params['num_filters2']

    # 划分训练集和验证集
    X_train, X_val, y_train_defect, y_val_defect, y_train_severity, y_val_severity = train_test_split(
        X_train_val, y_train_val_defect, y_train_val_severity, test_size=0.2, stratify=y_train_val_defect, random_state=42
    )

    # 创建数据加载器
    train_dataset = AugmentedDataset(X_train, y_train_defect, y_train_severity, augment=True)
    val_dataset = AugmentedDataset(X_val, y_val_defect, y_val_severity, augment=False)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    # 定义模型
    class CNN_Model(nn.Module):
        def __init__(self):
            super(CNN_Model, self).__init__()
            self.conv1 = nn.Conv1d(in_channels=3, out_channels=num_filters1, kernel_size=5, stride=1, padding=2)
            self.bn1 = nn.BatchNorm1d(num_filters1)
            self.pool1 = nn.MaxPool1d(kernel_size=2, stride=2)

            self.conv2 = nn.Conv1d(in_channels=num_filters1, out_channels=num_filters2, kernel_size=3, stride=1, padding=1)
            self.bn2 = nn.BatchNorm1d(num_filters2)
            self.pool2 = nn.MaxPool1d(kernel_size=2, stride=2)

            self.fc1 = nn.Linear(num_filters2 * 12, 128)  # 调整全连接层大小
            self.dropout = nn.Dropout(dropout_rate)
            self.fc2 = nn.Linear(128, 64)  # 增加复杂度

            self.fc_defect = nn.Linear(64, 3)  # 道路缺陷分类 (3 类)
            self.fc_severity = nn.Linear(64, 4)  # 严重程度分类 (4 类)

        def forward(self, x):
            x = x.permute(0, 2, 1)

            x = self.pool1(torch.relu(self.bn1(self.conv1(x))))
            x = self.pool2(torch.relu(self.bn2(self.conv2(x))))

            x = x.view(x.size(0), -1)
            x = torch.relu(self.fc1(x))
            x = self.dropout(x)
            x = torch.relu(self.fc2(x))

            defect_output = self.fc_defect(x)
            severity_output = self.fc_severity(x)

            return defect_output, severity_output

    model = CNN_Model()

    # 定义损失函数和优化器
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # 训练模型
    num_epochs = 20
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, defect_labels, severity_labels in train_loader:
            optimizer.zero_grad()
            defect_preds, severity_preds = model(inputs)
            loss_defect = criterion(defect_preds, defect_labels)
            loss_severity = criterion(severity_preds, severity_labels)
            loss = loss_defect + loss_severity
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

    # 在验证集上评估模型
    model.eval()
    all_defect_labels = []
    all_defect_preds = []
    all_severity_labels = []
    all_severity_preds = []

    with torch.no_grad():
        for inputs, defect_labels, severity_labels in val_loader:
            defect_preds, severity_preds = model(inputs)
            _, predicted_defects = torch.max(defect_preds, 1)
            _, predicted_severities = torch.max(severity_preds, 1)

            all_defect_labels.extend(defect_labels.cpu().numpy())
            all_defect_preds.extend(predicted_defects.cpu().numpy())
            all_severity_labels.extend(severity_labels.cpu().numpy())
            all_severity_preds.extend(predicted_severities.cpu().numpy())

    # 计算验证集上的准确率
    accuracy_defect = accuracy_score(all_defect_labels, all_defect_preds)
    accuracy_severity = accuracy_score(all_severity_labels, all_severity_preds)

    print(f"验证集上的道路缺陷分类准确率: {accuracy_defect * 100:.2f}%")
    print(f"验证集上的严重程度分类准确率: {accuracy_severity * 100:.2f}%")

# 使用最佳超参数重新训练和测试模型
train_and_test_with_best_params(study.best_params)


验证集上的道路缺陷分类准确率: 68.50%
验证集上的严重程度分类准确率: 65.50%
